In [ ]:
from HtmlCss import HtmlCSS
import ipywidgets as widgets
from IPython.display import display
import requests
from GlobalStatsPlots import GlobalStatsPlots
from LocalStatsPlots import LocalStatsPlots
import json

session = requests.Session()

employee_name_widget = widgets.Label(value='Bienvenido al Sistema Analítico Hotelero')
#section_titles = [(id, Titulo),
#                  (id, Titulo)]
section_titles = [("login", "Login Section"),
              ("local_statistics", "Local Statistics"),
              ("global_statistics", "Global Statistics"),
              ("create_records", "Create Records"),
              ("update_delete_records", "Update Delete Records")]

HtmlCSS().create_navBar(section_titles)

In [ ]:

display(employee_name_widget)


In [ ]:
def graficar_top_3_highest_pais_regular_employees(hid, json):#local stat
    with local_statistic_widget:
        LocalStatsPlots().graficar_top_3_highest_pais_regular_employees(session=session, json=json, hid=hid)
def graficar_top_5_clientes_jovenes_con_mas_reservaciones(hid, json):
    with local_statistic_widget:
        LocalStatsPlots().graficar_top_5_clientes_jovenes_con_mas_reservaciones(hid=hid, json=json, session=session)
def graficar_top_3_cuartos_reservados_con_menos_gtc_ratio(hid, json):
    with local_statistic_widget:
        LocalStatsPlots().graficar_top_3_cuartos_reservados_con_menos_gtc_ratio(hid=hid, json=json, session=session)
def graficar_total_reservation_by_room_type(hid, json):
    with local_statistic_widget:
        LocalStatsPlots().graficar_total_reservation_by_room_type(hid=hid, json=json, session=session)
def graficar_top_5_clients_received_the_most_discounts(hid, json):
    with local_statistic_widget:
        LocalStatsPlots().graficar_top_5_clients_received_the_most_discounts(hid=hid, json=json, session=session)
def graficar_top_3_rooms_with_least_reservation(hid, json):
    with local_statistic_widget:
        LocalStatsPlots().graficar_top_3_rooms_with_least_reservation(hid=hid, json=json, session=session)
def graficar_top_5_handicap_rooms_most_reserved(hid, json):
    with local_statistic_widget:
        LocalStatsPlots().graficar_top_5_handicap_rooms_most_reserved(hid=hid, json=json, session=session)


In [ ]:
global_statistic_widget = widgets.Output()#global stat
def mostrar_graficos_most_reservation():
    with global_statistic_widget:
        GlobalStatsPlots().mostrar_graficos_most_reservation(session=session)

def graficar_capacidad_hoteles(): #global stat
    with global_statistic_widget:
        GlobalStatsPlots().graficar_capacidad_hoteles(session=session)

def graficar_payment_method(): #global stat
    with global_statistic_widget:
        GlobalStatsPlots().graficar_payment_method(session=session)
def graficar_most_profit_month(): #globla stat
    with global_statistic_widget:
        GlobalStatsPlots().graficar_most_profit_month(session=session)
def graficar_top_3_chain_with_least_rooms(): #globla stat
    with global_statistic_widget:
        GlobalStatsPlots().graficar_top_3_chain_with_least_rooms(session=session)
def graficar_top_3_chains_with_the_highest_total_revenue():
    with global_statistic_widget:
        GlobalStatsPlots().graficar_top_3_chains_with_the_highest_total_revenue(session=session)
    

In [ ]:


# El widget Label para mostrar mensajes
mensaje_label_global_stats = widgets.Label()
# Aquí está tu función actualizada para verificar el acceso y mostrar gráficos si es necesario
def actualizar_interfaz_global_stats(cookies):
    if not cookies:
        mensaje_label_global_stats.value = "Necesitas hacer log in para ver la información en este área."
        # Asegurarse de que el área de gráficos esté vacía o escondida
        with global_statistic_widget:
            global_statistic_widget.clear_output()
    elif cookies.get('position', '') == 'Administrator':
        mensaje_label_global_stats.value = "Gráficos Globales"
        # Mostrar los gráficos porque el usuario es un administrador
        mostrar_graficos_most_reservation()
        graficar_capacidad_hoteles()
        graficar_payment_method()
        graficar_most_profit_month()
        graficar_top_3_chain_with_least_rooms()
        graficar_top_3_chains_with_the_highest_total_revenue()
    else:
        mensaje_label_global_stats.value = "No tienes permiso para ver esta sección."
        # Asegurarse de que el área de gráficos esté vacía o escondida
        with global_statistic_widget:
            global_statistic_widget.clear_output()

In [ ]:
local_statistic_widget = widgets.Output()
mensaje_label_local_statistic = widgets.Label()
dropdown_hotels = widgets.Dropdown(description='Hotel:')
button_show_stats = widgets.Button(description='Mostrar Estadísticas')

def actualizar_interfaz_local_stats(cookies):
    if not cookies:
        mensaje_label_local_statistic.value = "Necesitas hacer log in para ver la información en este área."
        # Asegurarse de que el área de gráficos esté vacía o escondida
        with local_statistic_widget:
            local_statistic_widget.clear_output()
    else:
        mensaje_label_local_statistic.value = "Estadisticas locales:"
        url_accessible_hotels = f"http://127.0.0.1:5000/employee/{session.cookies.get_dict()['eid']}/accessible-hotels"
        
        # Realizar la petición al endpoint
        response = session.get(url_accessible_hotels)
        # Depuración de la respuesta
        #print(f"Respuesta: {response.status_code}, Cuerpo: {response.text}")
        if response.status_code == 200:
            hotels_data = response.json()
            # Crear un dropdown para seleccionar el hotel
            dropdown_hotels.options = [(hotel['hname'], hotel['hid']) for hotel in hotels_data]

            # Botón para confirmar la selección y mostrar estadísticas
            def on_button_clicked(b):
                local_statistic_widget.clear_output(wait=True)
                graficar_top_3_highest_pais_regular_employees(hid=dropdown_hotels.value,json={'eid':session.cookies.get_dict()['eid']})
                graficar_top_5_clientes_jovenes_con_mas_reservaciones(hid=dropdown_hotels.value,json={'eid':session.cookies.get_dict()['eid']})
                graficar_top_3_cuartos_reservados_con_menos_gtc_ratio(hid=dropdown_hotels.value,json={'eid':session.cookies.get_dict()['eid']})
                graficar_total_reservation_by_room_type(hid=dropdown_hotels.value,json={'eid':session.cookies.get_dict()['eid']})
                graficar_top_5_clients_received_the_most_discounts(hid=dropdown_hotels.value,json={'eid':session.cookies.get_dict()['eid']})
                graficar_top_3_rooms_with_least_reservation(hid=dropdown_hotels.value,json={'eid':session.cookies.get_dict()['eid']})
                graficar_top_5_handicap_rooms_most_reserved(hid=dropdown_hotels.value,json={'eid':session.cookies.get_dict()['eid']})
            button_show_stats.on_click(on_button_clicked)
        else:
            print("Error al obtener la lista de hoteles accesibles.")

In [ ]:
# El widget Label para mostrar mensajes
update_delete_widget = widgets.Output()
mensaje_label_update_delete = widgets.Label()
dropdown_tables = widgets.Dropdown(description='Tables:')
dropdown_actions = widgets.Dropdown(description='Actions:')
# dropdown_fields = widgets.Dropdown(description='Search Record By:')
delete_widget = widgets.Output()
update_widget = widgets.Output()
id_widget = widgets.Output()
enter_id = widgets.Text(
                    value='',
                    placeholder='Enter ID of record',
                    description='ID',
                    disabled=False
                )
message = widgets.Label()
confirm_id = widgets.Button(description="Search Record") 
delete_record = widgets.Button(description="Delete Record")
update_record = widgets.Button(description="Update Record")
input_fields = []
save_record = widgets.Button(description="Save Record")







# Aquí está tu función actualizada para verificar el acceso y mostrar gráficos si es necesario
def actualizar_interfaz_update_delete(cookies):
    if not cookies:
        mensaje_label_update_delete.value = "Necesitas hacer log in para ver la información en este área."
        # Asegurarse de que el área de gráficos esté vacía o escondida
        with update_delete_widget:
            update_delete_widget.clear_output()
    elif cookies.get('position', '') == 'Administrator':
        with update_delete_widget:
            update_delete_widget.clear_output()
        
            mensaje_label_update_delete.value = "Update & Delete: Press  `Search Record` everytime the Action is changed"

            display(dropdown_tables,dropdown_actions,enter_id,confirm_id,message)

            url_tables = "http://127.0.0.1:5000/all/tables"
            response = session.get(url=url_tables)
            tables = response.json()
            dropdown_tables.options = [(table['table_name']) for table in tables]
            dropdown_actions.options = ['Update','Delete']
            def on_confirm_button_clicked(b):
                # update_delete_widget.clear_output(wait=True) 
                with id_widget:
                    id_widget.clear_output()
                    update_delete_widget.clear_output(wait=True)
                    delete_widget.clear_output()
                    update_widget.clear_output()
                    message.value = dropdown_tables.value
                    url = f"http://127.0.0.1:5000/{dropdown_tables.value}/{enter_id.value}"
                    record_request = session.get(url)
                    record = record_request.json()
                    message.value = f"{str(record)}"
                    if dropdown_actions.value == "Delete":
                        with delete_widget:
                            delete_widget.clear_output()
                            update_widget.clear_output()
                            display(delete_record)
                    elif dropdown_actions.value == "Update":
                        with update_widget:
                            delete_widget.clear_output()
                            update_widget.clear_output()
                            display(update_record)
            confirm_id.on_click(on_confirm_button_clicked)
        
            def on_delete_clicked(b):
                url = f"http://127.0.0.1:5000/{dropdown_tables.value}/{enter_id.value}"
                delete_request = session.delete(url)
                # message.value = f"{delete_request.json()}"
                with delete_widget:
                    print("Delete Succeded: ", delete_request)
            delete_record.on_click(on_delete_clicked)

            def on_update_clicked(b):
                with update_widget:
                    input_fields.clear()
                    url = f"http://127.0.0.1:5000/{dropdown_tables.value}/{enter_id.value}"
                    record_request = session.get(url)
                    record = record_request.json()    
                    key_url = f"http://127.0.0.1:5000/all/key/{dropdown_tables.value}"
                    key_request = session.get(key_url)
                    primary_key = key_request.json()

                    for key in record.keys():
                        desc = str(key)
                        val = str(record[key])
                        if primary_key[0]['attname'] == desc:
                            input_widget = widgets.Text(
                            value=val,
                            placeholder='',
                            description=desc,
                            disabled=True
                            )
                            display(input_widget)
                            input_fields.append(input_widget)
                        else:
                            input_widget = widgets.Text(
                            value=val,
                            placeholder='',
                            description=desc,
                            disabled=False
                            )
                            display(input_widget)
                            input_fields.append(input_widget)

                    
                    display(save_record)
                    # update_request = session.put(url,)
            update_record.on_click(on_update_clicked)

            def on_save_record(b):
                with update_widget:
                    update_widget.clear_output()
                    values = [field.value for field in input_fields]
                    url = f"http://127.0.0.1:5000/{dropdown_tables.value}/{enter_id.value}"
                    record_request = session.get(url)
                    record = record_request.json()   
                    dictionary = {key: value for key, value in zip(record.keys(),values)}
                    url = f"http://127.0.0.1:5000/{dropdown_tables.value}/{enter_id.value}"
                    # jsonnn = json.dumps(dictionary)
                    update_request = session.put(url,json=dictionary)
                    if update_request.status_code == 200:
                        print("Update Succeded: ", update_request)
                        message.value = ""
                    else:
                        print("Update Failed: ", update_request)
                        message.value = ""


            save_record.on_click(on_save_record)  
    else:
        mensaje_label_update_delete.value = "No tienes permiso para ver esta sección."
        mensafe_after_proceed=""
        with update_delete_widget:
            update_delete_widget.clear_output()
            id_widget.clear_output()
            delete_widget.clear_output()
            update_widget.clear_output()


In [ ]:


submit_login = widgets.Output()
# Widget para mostrar la información almacenada en las cookies

username_input = widgets.Text(
    value='',
    placeholder='Ingrese su nombre de usuario',
    description='Usuario:',
    disabled=False
)

password_input = widgets.Password(
    placeholder='Ingrese su contraseña',
    description='Contraseña:',
    disabled=False
)

login_button = widgets.Button(
    description='Iniciar sesión',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Enviar',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)



def on_login_button_clicked(b):
    with submit_login:
        submit_login.clear_output()
        # Aquí usarías `requests` para enviar los datos al servidor
        url = 'http://127.0.0.1:5000/login/in/frontend'
        data = {
            'username': username_input.value,
            'password': password_input.value
        }
        response = session.post(url, json=data, verify=False)  # `session` debería ser una instancia de `requests.Session()` ya creada
        
        # Ahora, maneja la respuesta
        if response.status_code == 200:
            print("Login exitoso")
            # Actualizar la información de las cookies
            employee_name_widget.value = "Bienvenido " + str(session.cookies.get_dict()['fname'] + " " + session.cookies.get_dict()['lname'] + " ") + "al Sistema Analítico Hotelero"
            # Aquí podrías actualizar el notebook o hacer algo con la respuesta
            actualizar_interfaz_global_stats(session.cookies.get_dict())
            actualizar_interfaz_local_stats(session.cookies.get_dict())
            actualizar_interfaz_update_delete(session.cookies.get_dict())
            actualizar_interfaz_add(session.cookies.get_dict())
            username_input.close()
            password_input.close()
            login_button.close()
        else:
            print("Usuario o contaseña incorrecta")


login_button.on_click(on_login_button_clicked)




In [ ]:
from ipywidgets import widgets, Button, Text, Label, Output, Dropdown
import requests

# Widgets globales
campos_output = Output()
save_record = Button(description="Guardar Registro")
add_message = Label()
mensaje_label_add = Label()
input_fields = []
table_selector = Dropdown(description='Selecciona:')

# Dropdowns específicos
habitacion_widget = Dropdown(description="rname", options=[
    'Standard', 'Standard Queen', 'Standard King', 'Double Queen', 'Double King', 'Triple King', 'Executive Family',
    'Presidential'])
tipo_widget = Dropdown(description="rtype")
capacidad_widget = Dropdown(description="capacity")
ishandicap_widget = Dropdown(description="ishandicap", options=[
    'true', 'false'])
position_widget = Dropdown(description="position", options=[
    'Administrator', 'Regular', 'Supervisor'])

# Interacción con API
def fetch_columns(table):
    """Fetch columns from the API for the specified table."""
    column_url = f"http://127.0.0.1:5000/all/columns/{table}"
    try:
        response = requests.get(column_url)
        response.raise_for_status()  # This will raise an exception for HTTP errors
        return response.json()
    except Exception as e:
        print(f"Failed to fetch columns: {e}")
        return None

def post_data(table_name, data):
    """Post data to the API for the specified table."""
    url = f"http://127.0.0.1:5000/{table_name}"
    try:
        response = requests.post(url, json=data)
        response.raise_for_status()
        add_message.value = 'Registro añadido exitosamente'
        return
    except Exception as e:
        add_message.value = 'Registro añadido exitosamente'
        return 
    
# Campos segun selección de tabla
def mostrar_campos(change):
        campos_output.clear_output()
        input_fields.clear()  
        with (campos_output):
            tabla = change.new if change else table_selector.value
            column_url = f"http://127.0.0.1:5000/all/columns/{tabla}"
            column_request = requests.get(column_url)
            if column_request.ok:
                columns = column_request.json()
                if columns:  # Check if columns are not empty
                    if tabla in ['roomunavailable', 'reserve']:
                        eid_label = Label('ID del empleado que llena la información:')
                        eid_widget = Text(value='', placeholder='Ingrese eid', description='eid', disabled=False)
                        input_fields.extend([eid_label, eid_widget])

                    for column in columns[1:]:
                        if tabla == 'login' and column["column_name"] in ['password']:
                            widget = widgets.Password(
                                placeholder='Ingrese su contraseña',
                                description='password',
                                disabled=False
                            )
                        elif tabla == 'chains' and column["column_name"] in ['springmkup', 'summermkup', 'fallmkup',
                                                                             'wintermkup']:
                            widget = widgets.FloatSlider(
                                value=1,  # Valor inicial
                                min=0.0,  # Valor mínimo
                                max=10.0,  # Valor máximo
                                step=0.1,  # Incremento decimal
                                description=column["column_name"],
                                orientation='horizontal',
                                readout=True,  # Muestra el valor numérico al lado del slider
                                readout_format='.1f',  # Formato de una cifra decimal
                            )
                        elif tabla == 'employee' and column["column_name"] in ['salary']:
                            widget = widgets.IntText(
                                value=0,  # Valor inicial
                                description='salary',  # Descripción o etiqueta del campo
                                disabled=False  # Habilitar o deshabilitar para la entrada del usuario
                            )
                        elif tabla == 'reserve' and column["column_name"] in ['guests']:
                            widget = widgets.IntText(
                                value=0,  # Valor inicial
                                description='guests',  # Descripción o etiqueta del campo
                                disabled=False  # Habilitar o deshabilitar para la entrada del usuario
                            )
                        elif tabla == 'roomdescription' and column["column_name"] in ['rname', 'rtype', 'capacity',
                                                                                      'ishandicap']:
                            if column["column_name"] == 'rname':
                                widget = habitacion_widget
                            elif column["column_name"] == 'rtype':
                                widget = tipo_widget
                            elif column["column_name"] == 'capacity':
                                widget = capacidad_widget
                            elif column["column_name"] == 'ishandicap':
                                widget = ishandicap_widget

                        elif tabla == 'employee' and column["column_name"] in ['position']:
                            if column["column_name"] == 'position':
                                widget = position_widget

                        elif tabla == 'roomunavailable' and column["column_name"] in ['startdate', 'enddate']:
                            if column["column_name"] == 'startdate':
                                widget = widgets.DatePicker(
                                    description='startdate',
                                    disabled=False
                                )
                            if column["column_name"] == 'enddate':
                                widget = widgets.DatePicker(
                                    description='enddate',
                                    disabled=False
                                )

                        else:
                            widget = Text(
                                value='',
                                placeholder=f'Ingrese {column["column_name"]}',
                                description=column["column_name"],
                                disabled=False
                            )
                        input_fields.append(widget)
                    display(*input_fields, save_record)
                    add_message.value = ' '
                    

# Función principal que construye la interfaz basada en las cookies
def actualizar_interfaz_add(cookies):
    if not cookies: 
        mensaje_label_add.value = "Necesitas hacer log in para ver la información en este área."
        display(mensaje_label_add)
        with campos_output:
            campos_output.clear_output()
        return
    
    position = cookies.get('position', '')
    mensaje_label_add.value = 'Acceso exitoso para '+position
    table_options = {
        'Administrator': [
            ('Login', 'login'),
            ('Cliente', 'client'),
            ('Empleado', 'employee'),
            ('Cadena', 'chains'),
            ('Hotel', 'hotel'),
            ('Habitación', 'room'),
            ('Descripción de habitación', 'roomdescription'),
            ('Habitación no disponible', 'roomunavailable'),
            ('Reservación', 'reserve')
        ],
        'Supervisor': [
            ('Habitación no disponible', 'roomunavailable')
        ],
        'Regular': [
            ('Reservación', 'reserve')
        ]
    }
    table_selector.options = table_options.get(position, [])
    table_selector.observe(mostrar_campos, names='value')

    def initial_display():
        mostrar_campos(None)

    # Cuando se guarda un record
    def on_save_record_clicked(b):
        mensaje_label_add.value = ""
        if not input_fields:
            add_message.value = "Por favor, seleccione una tabla y llene los datos."
            return

        table_name = table_selector.value
        data = {}
        eid_value = next((widget.value for widget in input_fields if widget.description == 'eid'), None)
        
        # Requieren ID empleado
        if table_name in ['roomunavailable', 'reserve']:
            data['eid'] = eid_value
        
        #Formato de fechas
        for widget in input_fields:
            if isinstance(widget, widgets.DatePicker):
                if widget.value is not None:
                    data[widget.description] = widget.value.strftime('%Y-%m-%d')
                else:
                    data[widget.description] = None
            else:
                data[widget.description] = widget.value

        result_message = post_data(table_name, data)
        add_message.value = result_message
        
    # Opciones dinamicas basadas en selección del nombre de la habitación
    def update_options(*args):
        nombre_habitacion = habitacion_widget.value
        if nombre_habitacion == 'Standard':
            tipo_widget.options = ['Basic', 'Premium']
            capacidad_widget.options = [1]
        elif nombre_habitacion == 'Standard Queen':
            tipo_widget.options = ['Basic', 'Premium', 'Deluxe']
            capacidad_widget.options = [1, 2]
        elif nombre_habitacion == 'Standard King':
            tipo_widget.options = ['Basic', 'Premium', 'Deluxe']
            capacidad_widget.options = [2]
        elif nombre_habitacion == 'Double Queen':
            tipo_widget.options = ['Premium', 'Deluxe', 'Suite']
            capacidad_widget.options = [4]
        elif nombre_habitacion == 'Double King':
            tipo_widget.options = ['Premium', 'Deluxe', 'Suite']
            capacidad_widget.options = [4, 6]
        elif nombre_habitacion == 'Triple King':
            tipo_widget.options = ['Deluxe', 'Suite']
            capacidad_widget.options = [6]
        elif nombre_habitacion == 'Executive Family':
            tipo_widget.options = ['Deluxe', 'Suite']
            capacidad_widget.options = [4, 6, 8]
        elif nombre_habitacion == 'Presidential':
            tipo_widget.options = ['Suite']
            capacidad_widget.options = [4, 6, 8]

    
    save_record.on_click(on_save_record_clicked)
    habitacion_widget.observe(update_options, 'value')
    initial_display()


In [ ]:
#Seccion para hacer log in

HtmlCSS().create_styled_section(section_titles[0][0], section_titles[0][1])
display(username_input, password_input, login_button, submit_login)


In [ ]:
#seccion para mostrar los local stats
HtmlCSS().create_styled_section(section_titles[1][0], section_titles[1][1])
display(mensaje_label_local_statistic, dropdown_hotels, button_show_stats, local_statistic_widget)
actualizar_interfaz_local_stats(session.cookies.get_dict())#Este solo ucurre al principio al entrar a la pagina y cuando se unde el boton de hacer login, se vuelve a llamar


In [ ]:
#seccion para mostrar los global stats
HtmlCSS().create_styled_section(section_titles[2][0], section_titles[2][1])
display(mensaje_label_global_stats, global_statistic_widget)       
actualizar_interfaz_global_stats(session.cookies.get_dict())#Este solo ucurre al principio al entrar a la pagina y cuando se unde el boton de hacer login, se vuelve a llamar




In [ ]:
HtmlCSS().create_styled_section(section_titles[3][0], section_titles[3][1])
display(table_selector, campos_output, add_message)
actualizar_interfaz_add(session.cookies.get_dict())



In [ ]:
#seccion para borrar y actualizar nuevos records
HtmlCSS().create_styled_section(section_titles[4][0], section_titles[4][1])
display(mensaje_label_update_delete,update_delete_widget,id_widget,delete_widget,update_widget)
actualizar_interfaz_update_delete(session.cookies.get_dict())




